# Basic Query Examples 

This notebook demonstrates some basic queries in the energy consumption model. 

 - Lat / Long or point query
 - Polygon query - full granular data
 - Polygon query - summary data


We also interpret the results of these queries and examine them in several ways for illustrative purposes. 


### Documentation

API developer documentation is located at 
(Stoplight.io) https://eguide.stoplight.io/docs/eguide-consumption-api/docs/_welcome.md 

 * Use the documentation to select the endpoint and the parameters. 
 * Run test queries immediately from your browser. 
 * automatically generate starter code snippets, in cURL, python, and more. 
 


### Authentication

We'll save our API key as an environment variable to use throughout these code examples

In [2]:
import os

In [3]:
# You can set it here if it isn't already set, like this: 
os.environ['eguideapikey'] = 'APIKEY_GOES_HERE'

In [4]:
# check that the environ variable is set
assert 'eguideapikey' in os.environ

# access it in python 
eguideapikey = os.environ['eguideapikey']

# show us what it is. 
print(eguideapikey)

APIKEY_GOES_HERE


### Using Curl


In [5]:
!curl --request GET \
  --url "https://api.eguide.io/v0/"

{"API Documentation":"https://stoplight.io/p/docs/gh/eguide/eguide-consumption-api","Valid endpoints":"https://api.eguide.io/v0/prediction","Query Parameters":["aoi(required)","resolution(optional)","summary_only(optional)"]}

In [6]:
!curl --request GET \
  --url 'https://api.eguide.io/v0/prediction?aoi=POINT(35.8843%204.0665)' \
  --header "x-api-key: $eguideapikey"

{
    "data": {
        "data_array": [
            {
                "cell_bbox": "POLYGON((35.8824639178345 4.06722456781251,35.8847096957719 4.06722456781251,35.8847096957719 4.06497878987503,35.8824639178345 4.06497878987503,35.8824639178345 4.06722456781251))",
                "cell_data": {
                    "classA_count": 5.0,
                    "classB_count": 0.0
                },
                "cell_id": 879,
                "model_output_id": 1
            }
        ],
        "number_of_cells": 1,
        "resolution": null,
        "summary_only": false
    },
    "data_summary": {
        "classA_pooled_prob_mean": 0.67,
        "classA_pooled_prob_std_dev": 0.1,
        "classA_total_count": 5,
        "classB_pooled_prob_mean": NaN,
        "classB_pooled_prob_std_dev": NaN,
        "classB_total_count": 0
    },
    "model_metadata": {
        "description": "Model output generated for project Alpha, client Acme\nIncludes 3 of the 14 requested regions\nSamples o

## Using Python (requests)

In [7]:
# We will use the requests python library
import requests

In [8]:
# Hello World query and response

url = 'https://api.eguide.io/v0'

response = requests.get(url)

print(response.text)

{"API Documentation":"https://stoplight.io/p/docs/gh/eguide/eguide-consumption-api","Valid endpoints":"https://api.eguide.io/v0/prediction","Query Parameters":["aoi(required)","resolution(optional)","summary_only(optional)"]}


In [9]:
# POINT prediction

url = "https://api.eguide.io/v0/prediction"

querystring = {"aoi":"POINT(35.8843 4.0665)"}

headers = {'x-api-key': eguideapikey}

response = requests.request("GET", url, headers=headers, params=querystring)

print(response.text)

{
    "data": {
        "data_array": [
            {
                "cell_bbox": "POLYGON((35.8824639178345 4.06722456781251,35.8847096957719 4.06722456781251,35.8847096957719 4.06497878987503,35.8824639178345 4.06497878987503,35.8824639178345 4.06722456781251))",
                "cell_data": {
                    "classA_count": 5.0,
                    "classB_count": 0.0
                },
                "cell_id": 879,
                "model_output_id": 1
            }
        ],
        "number_of_cells": 1,
        "resolution": null,
        "summary_only": false
    },
    "data_summary": {
        "classA_pooled_prob_mean": 0.67,
        "classA_pooled_prob_std_dev": 0.1,
        "classA_total_count": 5,
        "classB_pooled_prob_mean": NaN,
        "classB_pooled_prob_std_dev": NaN,
        "classB_total_count": 0
    },
    "model_metadata": {
        "description": "Model output generated for project Alpha, client Acme\nIncludes 3 of the 14 requested regions\nSamples o

In [13]:
# POLYGON prediction

url = "https://api.eguide.io/v0/prediction"

POLYGON_AOI = 'POLYGON((35.8851 4.0660, 35.8858 4.0644, 35.8831 4.0594, 35.8824 4.0658, 35.8851 4.0660 ))'

querystring = {
    "aoi": POLYGON_AOI,
    "summary_only": "True"
}

headers = {'x-api-key': eguideapikey}

response = requests.request("GET", url, headers=headers, params=querystring)

print(response.text)

{
    "data": {
        "number_of_cells": 4,
        "resolution": null,
        "summary_only": true
    },
    "data_summary": {
        "classA_pooled_prob_mean": 0.63,
        "classA_pooled_prob_std_dev": 0.08,
        "classA_total_count": 26,
        "classB_pooled_prob_mean": 0.55,
        "classB_pooled_prob_std_dev": 0.04,
        "classB_total_count": 12
    },
    "model_metadata": {
        "description": "Model output generated for project Alpha, client Acme\nIncludes 3 of the 14 requested regions\nSamples only, not intended for production\n",
        "model_output_id": 1,
        "model_run_date": "2020-05-30",
        "model_version": "0.1.1"
    },
    "request": {
        "AOI_coords": "'POLYGON((35.8851 4.0660, 35.8858 4.0644, 35.8831 4.0594, 35.8824 4.0658, 35.8851 4.0660 ))'",
        "AOI_type": "polygon",
        "other params": {},
        "requested_resolution": null,
        "summary_only": true
    }
}


In [12]:
# let's examine the aggregated data for the whole AOI 
data_summary = response.json()['data_summary']

request_info = response.json()['request']
for k, v in request_info.items():
    print (f"..{k:<20}: {v}")

print ("\nData Aggregated for the whole Polygon")
for k, v in data_summary.items():
    print (f"..{k:<20}: {v}")
        

..AOI_coords          : 'POLYGON((35.8851 4.0660, 35.8858 4.0644, 35.8831 4.0594, 35.8824 4.0658, 35.8851 4.0660 ))'
..AOI_type            : polygon
..other params        : {}
..requested_resolution: None
..summary_only        : False

Data Aggregated for the whole Polygon
..classA_pooled_prob_mean: 0.63
..classA_pooled_prob_std_dev: 0.08
..classA_total_count  : 26
..classB_pooled_prob_mean: 0.55
..classB_pooled_prob_std_dev: 0.04
..classB_total_count  : 12


In [14]:
# POLYGON prediction with cell data included

url = "https://api.eguide.io/v0/prediction"

POLYGON_AOI = 'POLYGON((35.8851 4.0660, 35.8858 4.0644, 35.8831 4.0594, 35.8824 4.0658, 35.8851 4.0660 ))'

querystring = {
    "aoi": POLYGON_AOI,
    "summary_only": "False" # <-- Set to False to include cell level data
}

headers = {'x-api-key': eguideapikey}

response = requests.request("GET", url, headers=headers, params=querystring)   

In [15]:
# let's examine the data_array
data_array = response.json()['data']['data_array']

n_cells_to_show = 4
for cell in data_array[:n_cells_to_show]:
    print(f"\nCell id: {cell['cell_id']}")
    
    for k, v in cell['cell_data'].items():
        print (f".. {k:<20}: {v}")


Cell id: 879
.. classA_count        : 5.0
.. classB_count        : 0.0

Cell id: 882
.. classA_count        : 9.0
.. classB_count        : 7.0

Cell id: 883
.. classA_count        : 2.0
.. classB_count        : 1.0

Cell id: 886
.. classA_count        : 10.0
.. classB_count        : 4.0


In [16]:
metadata = response.json()['model_metadata']

for k,v in metadata.items(): 
    print (f"{k:<20}: {v}")

description         : Model output generated for project Alpha, client Acme
Includes 3 of the 14 requested regions
Samples only, not intended for production

model_output_id     : 1
model_run_date      : 2020-05-30
model_version       : 0.1.1


### Using Geopandas and shapely object

In [30]:
### util functions

# We will use the requests python library
import requests

def get_data_summary(shapely_obj, apikey, resolution="250")-> list:
    """Function to return structure counts, uncertainity values for point or polygon shapely object
    Example:
        from shapely.geometry import Point, Polygon
        point = Point(0.0, 0.0) # shapely point object
        polygon = Polygon([(0, 0), (1, 1), (1, 0)])  # shapely polygon object

    Parameters
    ----------
    shapely_obj
        A shapely object representing a point or polygon
    apikey
        User's API key.

    Returns
    -------
    List
        Returns list of consisting of structure counts and probability {mean, standard deviation} from classes A and B.
    """
    url = "https://api.eguide.io/v0/prediction"

    lat = shapely_obj.coords[:][0][0]  # latitude coordinate

    lon = shapely_obj.coords[:][0][1]  # longitude coordinate

    querystring = {"aoi":f"POINT({lat} {lon})", "resolution": resolution} # query parameters

    headers = {'x-api-key': apikey} # API Key

    response = requests.request("GET", url, headers=headers, params=querystring)
    
    if (response.ok):
        classA_count = response.json()['data_summary']['classA_total_count']
        prob_mean_classA = response.json()['data_summary']['classA_pooled_prob_mean']
        prob_std_classA = response.json()['data_summary']['classA_pooled_prob_std_dev']
        classB_count = response.json()['data_summary']['classB_total_count']
        prob_mean_classB = response.json()['data_summary']['classB_pooled_prob_mean']
        prob_std_classB = response.json()['data_summary']['classB_pooled_prob_std_dev']
        return [classA_count, prob_mean_classA, prob_std_classA, classB_count, prob_mean_classB, prob_std_classB]
    else:
        # return None when response is empty
        return [None, None, None, None, None, None]


In [35]:
# Using 'get_data_summary' function on a shapely Point object
from shapely.geometry import Point, Polygon
p = Point(38.36387, -3.35493)
get_data_summary(shapely_obj=p, apikey=eguideapikey)

[0, nan, nan, 1, 0.65, nan]

In [7]:
# import multiple polygons or points from a file to geopandas dataframe
import geopandas as gpd # We will use the geopandas python library
path = 'path/to/file' # path to file with multiple polygon or point coordinates
dataframe = gpd.read_file(path)

In [26]:
%%time
# Apply 'get_data_summary' function to dataframe
dataframe[['classA_count','prob_mean_A','prob_std_A',
           'classB_count','prob_mean_B','prob_std_B']] = dataframe.apply(lambda x: 
                                                                         get_data_summary(x["geometry"]), 
                                                                         axis=1, result_type='expand')


CPU times: user 166 ms, sys: 21.1 ms, total: 187 ms
Wall time: 5.4 s


### environment information

In [17]:
!python --version

Python 3.7.4


In [18]:
requests.__version__

'2.22.0'